<img src="https://i1.wp.com/ayeai.xyz/site/wp-content/uploads/2019/09/cropped-AyeAI-logo-cropped-tm-150x150-v2.0.svg-1.png?fit=141%2C77&ssl=1" height="50" width="100">®<br>
# Data Analysis of Antibiogram
### AyeLearn by AyeAI™
##### Copyright (C) 2022 Abhishek Choudhary, Dr Srija Katta <br>
##### AyeAI Singularity Public License. No warranty. No Liability

# Initialization, Training and Test

#### Disclaimer: This is only an academic exercise and must notbe used in a clinical setup with real patients, unless duly verified, tested and approved by drug control authorities

In [ ]:
%%bash
#install python3
#sudo apt-get install python3 -y

#install python3-pip
sudo apt-get -y install python3-pip




In [ ]:
%%bash
#install numpy
python3 -m pip install --user numpy

#install scipy
python3 -m pip install --user scipy




In [ ]:
%%bash
#install scikit-learn using pip 
pip3 install -U scikit-learn
#check and verify scikit-learn
python3 -m pip show scikit-learn



In [ ]:
%%shell
if [ -x dataset ]; then mv dataset dataset.$(date +%s); fi
git clone https://github.com/srija-ieee/MLbased_analysis dataset


In [ ]:
import pandas as pd
import numpy as np
din = pd.read_csv("dataset/genloc_isolate.csv")
df = pd.DataFrame(din)
corr = df.corr()
corr
x=np.asarray(df.genloc)
y=np.asarray(df.isolate)
#from sklearn.linear_model import LogisticRegression
from sklearn import linear_model as lm
#model=lm.Perceptron()
model=lm.LogisticRegression()
x=x.reshape(-1,1)
model.fit(x,y)
model.predict(x)
#model.predict_proba(x)

In [23]:
!cat dataset/genloc_isolate.csv | grep -v 81 | grep -v 150 | grep -v 84 > dataset/genloc_isolate.min.csv

In [ ]:
import pandas as pd
import numpy as np
din = pd.read_csv("dataset/genloc_isolate.min.csv")
df = pd.DataFrame(din)
corr = df.corr()
corr
x=np.asarray(df.genloc)
y=np.asarray(df.isolate)
#from sklearn.linear_model import LogisticRegression
from sklearn import linear_model as lm
#model=lm.Perceptron()
model=lm.LogisticRegression()
x=x.reshape(-1,1)
model.fit(x,y)
model.predict(x)
#model.predict_proba(x)

In [ ]:
model.predict_proba(x)[2]

In [ ]:
!cat dataset/isolates.coding | tr ';' '\n' | awk -F',' '{printf "s/%s/%s/g;",$4,$2};END {printf "\n"}' > dataset/isolates.reverse
rev_filt = !cat dataset/isolates.reverse
preds =   model.predict(x)
diff_probs = model.predict_proba(x)
p=0
for n in preds:
  p=p+1
  print("Patient ",p,": ",end=' ')
  cmd_str="echo " + str(n) + " | sed " + """ '""" + rev_filt[0] + """' """
  !{cmd_str}
  print("Differential probabilities")
  print(diff_probs[p-1])


# CDSS (based on predictive model)

#### Disclaimer: For academic usage only. Not to be used with real patients in clinical settings. The predictions may be biased based on skews in the training dataset, and are subject to errors of calculation in the underlying framework used. No warranty. No Liability.

In [143]:
#@title UTI CDSS (isolate prediction based on gender and location) { run: "auto", vertical-output: true }
Name = "PN" #@param {type:"string"}
Gender = "Female" #@param ["Male", "Female"]
Age = 61 #@param {type:"slider", min:5, max:100, step:1}
Location = "Semi Urban" #@param ["Urban", "Semi Urban", "Rural"]

from IPython import display

if Gender == "Male":
  g=7
else:
  g=11

if Location == "Urban":
  l = 85
else:
  if Location == "Rural":
    l = 51
  else:
    l = 68

x=[[g*l]]
ppre=model.predict(x)[0]
cmd_str="echo " + str(ppre) + " | sed " + """ '""" + rev_filt[0] + """' """
#print("Most likely isolate expected")
res=!{cmd_str}
#print(res[0])

with open("report.html","w") as fl:
  fl.write("""<img src="https://i1.wp.com/ayeai.xyz/site/wp-content/uploads/2019/09/cropped-AyeAI-logo-cropped-tm-150x150-v2.0.svg-1.png?fit=141%2C77&ssl=1" height="50" width="100"></img>®""")
  fl.write("<h1>AyeVH Project Internship</h1>")
  fl.write("<h2>CDSS Report</h2>")
  fl.write("<h3>Patient: " + Name)
  fl.write("</h3><b>Gender: " + Gender)
  fl.write("<br>Age: " + str(Age))
  fl.write("<br>Location: " + Location)
  fl.write("</h3>Genloc index = " + str(g*l))
  fl.write("<br><br>Most likely isolate expected: <h3>")
  fl.write(res[0])
  fl.write("</b></h3><br>Differential probabilties (refer pathogen encoding)<br>")
  l=0
  for n in model.predict_proba(x)[0]:
    fl.write(str(n))
    l+=1
    fl.write(" ")
    if l % 3 == 0:
      fl.write("<br>")
  fl.write("<br><br><small>Disclaimer:<br>")
  fl.write("This is for academic purposes only.")
  fl.write("The results must NOT be used for real patients without due testing, verification and authorization by drug control authorities.")
  fl.write("This model was developed as a part of Project Internship with AyeAI")
  fl.write('<br><br>Visit <a href="https://ayeai.xyz">AyeAI site</a> for more learning opportunities in <b>Artificial Intelligence and Machine Learning in Healthcare</b></small>')

html_rep=!cat report.html

html_str=""
for n in html_rep:
  html_str+=n

display.HTML(html_str)